In [1]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

import numpy as np
# from tensorflow.lite.python.lite import Interpreter
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

In [3]:
# https://inpages.tistory.com/155
# to avoid GPU full memory error
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
#         tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
    
#         tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1800)])
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

    except RuntimeError as e:
        # Visible devices must be set at program startup
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
# Load the TFLite model and allocate tensors.
# interpreter = Interpreter(model_path="model.tflite")
interpreter = tf.lite.Interpreter(model_path="trained_model_lite_200904.tflite")
interpreter.allocate_tensors()

https://www.tensorflow.org/lite/guide/inference#load_and_run_a_model_in_python

In [5]:
def normalize(input_image):
    input_image = (input_image / 255.0)
    return input_image

In [6]:
def preprocess(input_image):
#     image = tf.image.decode_jpeg(input_image)
    image = tf.cast(input_image, tf.float32)
    image = normalize(image)

    return image

In [7]:
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image)
    image = tf.cast(image, tf.float32)
    image = normalize(image)

    # image = image.numpy()
    return image

In [8]:
image_free = load('./free_1.jpg')
image_block = load('./block_1.jpg')

Executing op ReadFile in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DecodeJpeg in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Cast in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RealDiv in device /job:localhost/replica:0/task:0/device:GPU:0


https://github.com/tensorflow/examples/blob/master/lite/examples/object_detection/raspberry_pi/detect_picamera.py

In [9]:
def set_input_tensor(interpreter, image):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
#   print(input_tensor)
    input_tensor[:, :] = image
#   print(input_tensor)
#   print(input_tensor.shape)

In [10]:
def get_output_tensor(interpreter, index):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
    return tensor

In [11]:
# tensor_index = interpreter.get_input_details()[0]['index']
# output_index = interpreter.get_output_details()[index]['index']
def interpreter_calculate(interpreter, image):
    set_input_tensor(interpreter, image)
    interpreter.invoke()

    return get_output_tensor(interpreter,0)

In [12]:
print(interpreter_calculate(interpreter, image_free))

[3.8452375e-09 1.0000000e+00]


In [13]:
print(interpreter_calculate(interpreter, image_block))

[9.9999988e-01 1.3733975e-07]


In [14]:
# import traitlets
# from IPython.display import display
# import ipywidgets.widgets as widgets
# from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

In [15]:
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    x = tf.expand_dims(x, 0)
    y = interpreter_calculate(interpreter, x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
#     y = F.softmax(y, dim=1)
#     print(y)
#     prob_blocked = float(y.flatten()[0])

#     blocked_slider.value = prob_blocked
    blocked_slider.value = y[0]
    
#     if prob_blocked < 0.5:
#         robot.forward(0.02)
#     else:
#         robot.left(0.003)
#         time.sleep(0.8)
    
#     time.sleep(0.1)
        
update({'new': camera.value})  # we call the function once to intialize

Executing op ExpandDims in device /job:localhost/replica:0/task:0/device:GPU:0


In [16]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera